In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob

# Load data

In [ ]:
# read HAND data to generate drainage paths 
hand30_100 = ee.ImageCollection('users/gena/global-hand/hand-100')
hand30_1000 = ee.Image('users/gena/GlobalHAND/30m/hand-1000')
hand30_5000 = ee.Image('users/gena/GlobalHAND/30m/hand-5000')
srtm = ee.Image("USGS/SRTMGL1_003")

# Map.addLayer(hand30_5000,{},"hand",False)

## exclude SWBD water
swbd = ee.Image('MODIS/MOD44W/MOD44W_005_2000_02_24').select('water_mask')
swbdMask = swbd.unmask().Not().focal_median(1) #.focal_max(5) #.add(0.2);

## select HAND height by flow accumuation levels
thresh = 0
threshStr=str(thresh)
FlowAccu = hand30_1000 # options: hand30_100, hand30_1000, hand30_5000
FA = 1000 # options: 100, 1000, 5000
FAstr=str(FA)
HANDmFA = FlowAccu.lte(thresh).focal_max(1).focal_mode(2, 'circle', 'pixels', 5).mask(swbdMask)
HANDmFA = HANDmFA.mask(HANDmFA)
HANDscale =HANDmFA.projection().nominalScale()
HANDCRS =HANDmFA.projection().crs()
# Map.addLayer(HANDmFA, {'palette':['0020ff'], 'opacity':0.8},'water (HAND < '+threshStr+'m, FA'+FAstr+')', False);


In [ ]:
# read and map HydroSHEDS river data
dataset = ee.FeatureCollection('WWF/HydroSHEDS/v1/FreeFlowingRivers');
# Paint "RIV_ORD" (river order) value to an image for visualization.
datasetVis = ee.Image().byte().paint(dataset, 'RIV_ORD', 2);
visParams = {'min': 1,'max': 10,'palette': ['08519c', '3182bd', '6baed6', 'bdd7e7', 'eff3ff']};

# Map.addLayer(datasetVis, visParams, 'Free flowing rivers');
#Map.addLayer(dataset, {}, 'Free flowing rivers', False);

In [ ]:
# Read surface water occurance
water = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select(['occurrence']).gte(50);
# water = water.reproject(**{'crs':HANDCRS,'scale':HANDscale})##'crsTransform':esa_PROJ.transform,

#visualization = {'bands': ['occurrence'],'min': 50,'max': 100,'palette':['black'] };['ffffff', 'ffbbbb', '0000ff']
# Map.addLayer(water, {'palette':['black']}, 'JRC Water occurrence',False);
#var watermask = water.select('occurrence').gte(90).where(water.select('occurrence'),0).unmask(1).selfMask()


In [ ]:
#Create combined water layer and mask
#combWater = ee.ImageCollection([HANDmFA,water]).mosaic()
combWater = HANDmFA.blend(water).gt(0)
combWaterScale =combWater.projection().nominalScale()
combWaterCRS =combWater.projection().crs()

# Map.addLayer(combWater, {'palette':['blue']}, 'Combined water',False);

In [ ]:
# Buffer waterways by riparian zone definitions 
Distance = combWater.unmask().distance(ee.Kernel.euclidean(200, 'meters'))
halfpixel = combWaterScale.multiply(0.5)
## https://doi.org/10.1016/j.jenvman.2019.109391
nutrientBuffer = Distance.updateMask(Distance.lte(ee.Number(3).subtract(halfpixel)))#.updateMask(Distance.gt(0))
floraBuffer = Distance.updateMask(Distance.lte(ee.Number(24).subtract(halfpixel)))#.updateMask(Distance.gt(0))
birdBuffer = Distance.updateMask(Distance.lte(ee.Number(144).subtract(halfpixel)))#.updateMask(Distance.gt(0))
riparianBuffer = birdBuffer
riparianMask = riparianBuffer.updateMask(riparianBuffer.gt(0))
# riparianMask = riparianMask.reproject(**{'crs':greenCRS,'scale':greenScale})##'crsTransform':esa_PROJ.transform,
riparianMaskScale = riparianMask.projection().nominalScale()

# Map.addLayer(Distance,{'min': 0, 'max': 200},"Distance",False)
#Map.addLayer(nutrientBuffer,{'min': 0, 'max': 200},"nutrientBuffer")
#Map.addLayer(floraBuffer,{'min': 0, 'max': 200},"floraBuffer")
#Map.addLayer(riparianBuffer,{'min': 0, 'max': 200},"riparianBuffer",False)
# Map.addLayer(riparianMask,{'min': 0, 'max': 200},"riparianMask")

In [ ]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

# Extract riparian areas by city

In [ ]:
for i in range(0,len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # Download ee.Image of albedo as GeoTIFF to Google Drive 
    geemap.ee_export_image_to_drive(
        riparianMask.toByte(), # use toByte() to reduce file size
        description = boundary_id + '-RiparianBuffer',
        folder='data', 
        scale=riparianMaskScale, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )

# Upload in aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

# specify bucket name
bucket_name = 'cities-cities4forests' 

In [ ]:
out_dir = os.getcwd()

In [ ]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id + '-RiparianBuffer.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/flooding/riparian/v_0/' + boundary_id + '-RiparianBuffer.tif',
                               ExtraArgs={'ACL':'public-read'})

# Visualize data

In [ ]:
## create map
Map = geemap.Map(height="350px")
Map

In [ ]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(boundary_geo_ee, zoom=8)

In [ ]:
Map.addLayer(greenmask),
             {'min':0, 'max':0.5, 'palette':['white','#006400']},
             'Vegetation cover',True,1)